In [ ]:
# Find duplicates using statistics for each image (https://pillow.readthedocs.io/en/stable/reference/ImageStat.html).
# Worked well in the sense that all images it found were real duplicates.
# Took almost 2 h to run locally.
from tqdm import tqdm
import os
from PIL import Image, ImageStat
from pathlib import Path



image_paths = list(Path('../data/').glob('train/**/*.png'))
image_paths += list(Path('../data/').glob('val/**/*.png'))

duplicate_files = []

#this could be improved considerable by storing mean of each image in a dictionary and running a lookup instead.
for file_org in tqdm(image_paths):
    if not file_org in duplicate_files:
        pixels_mean_org = ImageStat.Stat(Image.open(str(file_org))).mean
        for file_check in image_paths:
            if file_check != file_org:
                pix_mean_check = ImageStat.Stat(Image.open(str(file_check))).mean
                if pixels_mean_org == pix_mean_check:
                    duplicate_files.append((file_org))
                    duplicate_files.append((file_check))

list(dict.fromkeys(duplicate_files))

In [12]:
len(duplicate_files)

98

In [ ]:
# Visuallly inspect the results
from IPython.display import display

for f in duplicate_files:
    display(Image.open(str(f)))

In [14]:
# Save results
with open('../duplicate_imgs.txt','w+') as f:
    f.writelines('\n'.join([str(_) for _ in duplicate_files]))

In [ ]:
"""
METHOD BELOW DOES NOT WORK. IT MARKED IMAGES AS DUPLICATES THAT WEREN'T DUPLICATES. 


STILL INTERESTING, SO I KEEP IT :)

"""

In [ ]:
import cv2

def dhash(image, hash_size=8):
    #https://www.pyimagesearch.com/2020/04/20/detect-and-remove-duplicate-images-from-a-dataset-for-deep-learning/https://www.pyimagesearch.com/2020/04/20/detect-and-remove-duplicate-images-from-a-dataset-for-deep-learning/
	# convert the image to grayscale and resize the grayscale image,
	# adding a single column (width) so we can compute the horizontal
	# gradient
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	resized = cv2.resize(gray, (hash_size + 1, hash_size))
	# compute the (relative) horizontal gradient between adjacent
	# column pixels
	diff = resized[:, 1:] > resized[:, :-1]
	# convert the difference image to a hash and return it
	return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

In [ ]:
from pathlib import Path


# grab the paths to all images in our input dataset directory and
# then initialize our hashes dictionary
image_paths = list(Path('../data/').glob('train/**/*.png'))
image_paths += list(Path('../data/').glob('val/**/*.png'))

print(f"Deduping {len(image_paths)} images")
#image_paths = Path('../data/val')


hashes = {}
# loop over our image paths
for image_path in image_paths:
	# load the input image and compute the hash
	image = cv2.imread(str(image_path))
	h = dhash(image)
	# grab all image paths with that hash, add the current image
	# path to it, and store the list back in the hashes dictionary
	p = hashes.get(h, [])
	p.append(image_path)
	hashes[h] = p

cnt=0
for k,v in hashes.items():
	if len(v)>1:
		cnt+=len(v)
print(f"Found {cnt} duplicate images")
	

In [ ]:
#from matplotlib import pyplot as plt
import time

from PIL import Image
import cv2 
from IPython.display import display

#Manually check duplicates
for k,v in hashes.items():
	if len(v)>1:
		print(len(v))
		print(f"Dup {v}")
		# for img in v:
		# # 	plt.imshow(cv2.imread(str(img)))
		# # # plt.imshow(gray)
		# # # plt.imshow(gray)
		# # #plt.title('my picture')
		# # plt.show()

		# 	img = cv2.imread(str(img)) # with the OpenCV function imread(), the order of colors is BGR (blue, green, red).
		# 	# In Pillow, the order of colors is assumed to be RGB (red, green, blue).
		# 	# As we are using Image.fromarray() of PIL module, we need to convert BGR to RGB.
		# 	img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
		# 	display(Image.fromarray(img))
		
		# time.sleep(60)









In [ ]:
import os
hashes
#remove duplicate images from train-clean
data_dir=Path('../data/')
cnt_train=0
cnt_val=0
for k,v in hashes.items():
	if len(v)>1:
                for dup in v[1:]:
                        for match in (data_dir/'train-clean').glob(f'**/{dup}'):
                                os.remove(match)
                                cnt_train+=1
                        for match in (data_dir/'val-clean').glob(f'**/{dup}'):
                                os.remove(match)
                                cnt_val+=1
print(f"Removed {cnt_train} images from training set\nRemoved {cnt_val} images from validation set")

        
